Copyright 2024 Lehi Gracia

### Save?

In [31]:
save_to_mongo = False


### Markets

In [32]:

import warnings
warnings.filterwarnings('ignore')

import subprocess, os
from yfinance_utils import list_utils, file_utils, constants
import yfinance 
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
from finta import TA

if save_to_mongo:
    mongod = subprocess.Popen(
        ['mongod', '--dbpath', os.path.expanduser(os.getenv('MONGODB_PATH')), '--logpath', os.path.expanduser(os.getenv('MONGODB_LOG'))]
    )

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
symbols = list_utils.get_markets()
names = list_utils.markets
    
fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(title="MARKETS", title_x=0.5, width=600, height=400)

data = yfinance.download(symbols, period='1y')
gains = {}
for symbol in symbols:
    df1 = pd.DataFrame()
    df1 = data.loc[:,(slice(None),symbol)]
    df1.columns = COLUMNS
    df1['sma'] = TA.SMA(df1, period=50)
    df1 = df1[50:]
    df1.reset_index(inplace=True)

    fig.add_trace(
        go.Scatter(y=df1['Close'], x=df1['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig.add_trace(
        go.Scatter(y=df1['sma'], x=df1['Date'], name=f'sma'),
        secondary_y=False
    )

    temp_1 = df1['Close'].iloc[-1]
    temp_2 = df1['Close'].iloc[-30]
    gains[names[symbol]] = round(((temp_1 - temp_2) / temp_2 * 100),2)
fig.show()


[*********************100%***********************]  3 of 3 completed


### Implied Volaility

In [33]:
import yfinance
from finta import TA
from plotly.subplots import make_subplots
_t = yfinance.Ticker('^VIX')

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
_d = _t.history(period='1y')
_d['sma'] = TA.SMA(_d, period=30)

_f=make_subplots(specs=[[{"secondary_y": True}]])
_f.update_layout(title=f"Market Volatility (VIX)", title_x=0.5, width=600, height=400)
_d.reset_index(inplace=True)
_f.add_trace(
    go.Scatter(y=_d['Close'], x=_d['Date'], name="VIX"),
    secondary_y=False
)
_f.add_trace(
    go.Scatter(y=_d['sma'], x=_d['Date'], name=f'sma'),
    secondary_y=False
)
_f.show()

### Sectors

In [34]:

from yfinance_utils import list_utils
from datetime import datetime

symbols = list_utils.get_sectors()
names = list_utils.sectors
COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
NUMBER_OF_SECTORS = 12


top = list_utils.get_top_performers(symbols, size=NUMBER_OF_SECTORS)
_list = [i[0] for i in top]

data = yfinance.download(_list, period='1y')
for symbol in _list:
    fig2=make_subplots(specs=[[{"secondary_y": True}]])
    fig2.update_layout(title=f"{NUMBER_OF_SECTORS} SECTORS", title_x=0.5, width=600, height=400)
    _d2 = pd.DataFrame()
    _d2 = data.loc[:,(slice(None),symbol)]
    _d2.columns = COLUMNS
    _d2['sma'] = TA.SMA(_d2, period=50)
    _d2.reset_index(inplace=True)
    fig2.add_trace(
        go.Scatter(y=_d2['Close'], x=_d2['Date'], name=names[symbol]),
        secondary_y=False
    )
    fig2.add_trace(
        go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
        secondary_y=False
    )
    fig2.show()

today = datetime.today().strftime('%Y-%m-%d')
print("PERFORMANCE IN THE LAST 30 DAYS:")
for symbol, value in top:
    if save_to_mongo:
        file_utils.save_to_mongo({"DATE":today, "TICK":symbol, "PERFORMANCE":value, "SECTOR":names[symbol]}, "top_sector_performer")



[*********************100%***********************]  12 of 12 completed
[*********************100%***********************]  12 of 12 completed


PERFORMANCE IN THE LAST 30 DAYS:


###  Getting best performing stocks in each sectors

In [35]:

from yfinance_utils import list_utils
import time

NUMBER_OF_TOP_PERFORMERS_IN_SECTORS = 2
_sym = {}
_tp_list = {}
for symbol, value in top:
    _l = []
    for s in list_utils.SECTORS[symbol].keys():
        _l.append(s)
    _tp = list_utils.get_top_performers(_l, size=NUMBER_OF_TOP_PERFORMERS_IN_SECTORS)
    _tp_list.update(dict(_tp))
    _l = [i[0] for i in _tp]
    _sym[symbol] = _l

for _s in _sym:
    time.sleep(5)
    data = yfinance.download(_sym[_s], period='1y')
    for symbol in _sym[_s]:
        _f=make_subplots(specs=[[{"secondary_y": True}]])
        _f.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS_IN_SECTORS} from {names[_s]}", title_x=0.5, width=600, height=400)
        _d2 = pd.DataFrame()
        _d2 = data.loc[:,(slice(None),symbol)]
        _d2.columns = COLUMNS
        _d2['sma'] = TA.SMA(_d2, period=50)
        _d2.reset_index(inplace=True)
        _f.add_trace(
            go.Scatter(y=_d2['Close'], x=_d2['Date'], name=f"{symbol}: {_tp_list[symbol]} %"),
            secondary_y=False
        )
        _f.add_trace(
            go.Scatter(y=_d2['sma'], x=_d2['Date'], name=f'sma'),
            secondary_y=False
        )
        _f.update_yaxes(side='right')
        price = f"Date: {today}, Open:{round(_d2['Open'].iloc[-1],2)}, High:{round(_d2['High'].iloc[-1],2)}, Low:{round(_d2['Low'].iloc[-1],2)}, Close: {round(_d2['Close'].iloc[-1],2)}"
        _f.add_annotation(xref='x domain',
                        yref='y domain',
                        x=0.01,
                        y=1,
                        text=price, 
                        showarrow=False,
                        font=dict(size=8),
                        row=1, col=1)
        _f.update_layout(legend={"title_font_family":"Arial", "font":{"size":8}})
        _f.show()

if save_to_mongo:
    _sym['DATE'] = datetime.today().strftime('%Y-%m-%d')
    file_utils.save_to_mongo(_sym, "top_performers")

[*********************100%***********************]  50 of 50 completed
[*********************100%***********************]  69 of 69 completed
[*********************100%***********************]  21 of 21 completed
[*********************100%***********************]  54 of 54 completed
[*********************100%***********************]  22 of 22 completed
[*********************100%***********************]  73 of 73 completed
[*********************100%***********************]  61 of 61 completed
[*********************100%***********************]  78 of 78 completed
[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  37 of 37 completed
[*********************100%***********************]  28 of 28 completed
[*********************100%***********************]  31 of 31 completed
[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


...

[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


[*********************100%***********************]  2 of 2 completed


In [36]:

print('The END')

The END
